In [ ]:
# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import time;

from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate
from src.data.load_data import *

layers = 2
units_in_layers = [50, 5]
units_first_layer = 100

# load data // load_data.py
df = load_dataset(filename='../data/Jester-Dataset-ratings.csv')
user_ids, joke_ids, ratings = get_data(df=df, batch_size=20000)


# neural network architecture // model.py

# input layers
user_input = Input(shape=(1,), dtype='float64')
joke_input = Input(shape=(1,), dtype='float64')

# embedding layers
user_embedding = Embedding(
    input_dim=len(user_ids),
    output_dim=int(units_first_layer / 2),
    input_length=1
)

joke_embedding = Embedding(
    input_dim=len(joke_ids),
    output_dim=int(units_first_layer / 2),
    input_length=1
)


# flatten embeddings
user_latent = Flatten()(user_embedding(user_input))
joke_latent = Flatten()(joke_embedding(joke_input))

# concatenate user and joke embeddings

final_vector = concatenate([user_latent, joke_latent])


# hidden layers
for i in range(1, layers):
    layer = Dense(units=units_in_layers[i], activation='relu')
    final_vector = layer(final_vector)

# output layer
output = Dense(units=1, activation='tanh')(final_vector)

# Training routine // train.py

model = Model(
    inputs=[user_input, joke_input],
    outputs=output
)

model.compile(
    optimizer='adam', 
    loss='mean_squared_error', 
    metrics=['accuracy']
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("../logs", str(time.time())),
    histogram_freq=1)

model.fit(
    x=[np.array(user_ids), np.array(joke_ids)],
    y=np.array(ratings),
    batch_size=100, 
    epochs=50,
    callbacks=[tensorboard_callback],
    validation_split=0.2
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 5s 334us/sample - loss: 33.1724 - accuracy: 0.0021 - val_loss: 33.0530 - val_accuracy: 7.5000e-04
Epoch 2/50
  100/16000 [..............................] - ETA: 3s - loss: 30.3647 - accuracy: 0.0000e+00